<a href="https://colab.research.google.com/drive/1GWsEqIDuIt3HR6ljwyzuA82DpXWQ5e9o" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **MRI classification on morphometry data**

#### 1. Introduction

In this notebook we will do conventional morphometry analysis for gender-related morphometric sighns search.


**Our goal will be to build a network for MEN and WOMEN brain classification, to explore gender influence on brain structure and find gender-specific biomarkers.**

We will use the data from https://db.humanconnectome.org/data/projects/HCP_1200.

*Proceeding with this Notebook you confirm your personal acess [to the data](https://www.humanconnectome.org/study/hcp-young-adult/document/1200-subjects-data-release). 
 And your agreement on data [terms and conditions](https://www.humanconnectome.org/study/hcp-young-adult/data-use-terms).*


You can use plug-and-play grid seach pipleine from https://github.com/kondratevakate/neurobot

2. Mounting Google Drive to Collab Notebook. You should go with the link and enter your personal authorization code:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**!!! Add a shortcut** with "Add a shortcut" with the `data` repository from here https://drive.google.com/drive/folders/1zajXzS7dMwaBrhauhvRph2La1uOgP6jN?usp=sharing

After adding a shotcut change this data dir to yours own files location in Google drive and give a path to `seminars/anat` folder:

In [16]:
data_dir = '/content/drive/My Drive/Skoltech Neuroimaging/NeuroML2020/data/seminars/anat/'

### Importing the data

Importing `unrestricted_freesurfer.csv` dataset from https://db.humanconnectome.org/data/projects/HCP_1200.

In [17]:
import pandas as pd
data = pd.read_csv(data_dir + 'unrestricted_hcp_freesurfer.csv')

In [18]:
data.head()

,Subject,Gender,Release,FS_InterCranial_Vol,FS_BrainSeg_Vol,FS_BrainSeg_Vol_No_Vent,FS_BrainSeg_Vol_No_Vent_Surf,FS_LCort_GM_Vol,FS_RCort_GM_Vol,FS_TotCort_GM_Vol,FS_SubCort_GM_Vol,FS_Total_GM_Vol,FS_SupraTentorial_Vol,FS_SupraTentorial_Vol_No_Vent,FS_SupraTentorial_No_Vent_Voxel_Count,FS_L_WM_Vol,FS_R_WM_Vol,FS_Tot_WM_Vol,FS_Mask_Vol,FS_BrainSegVol_eTIV_Ratio,FS_MaskVol_eTIV_Ratio,FS_LH_Defect_Holes,FS_RH_Defect_Holes,FS_Total_Defect_Holes,FS_L_LatVent_Vol,FS_L_InfLatVent_Vol,FS_L_Cerebellum_WM_Vol,FS_L_Cerebellum_Cort_Vol,FS_L_ThalamusProper_Vol,FS_L_Caudate_Vol,FS_L_Putamen_Vol,FS_L_Pallidum_Vol,FS_3rdVent_Vol,FS_4thVent_Vol,FS_BrainStem_Vol,FS_L_Hippo_Vol,FS_L_Amygdala_Vol,FS_CSF_Vol,FS_L_AccumbensArea_Vol,FS_L_VentDC_Vol,...,FS_L_Superiortemporal_CurvInd,FS_L_Supramarginal_CurvInd,FS_L_Frontalpole_CurvInd,FS_L_Temporalpole_CurvInd,FS_L_Transversetemporal_CurvInd,FS_L_Insula_CurvInd,FS_R_Bankssts_CurvInd,FS_R_Caudalanteriorcingulate_CurvInd,FS_R_Caudalmiddlefrontal_CurvInd,FS_R_Cuneus_CurvInd,FS_R_Entorhinal_CurvInd,FS_R_Fusiform_CurvInd,FS_R_Inferiorparietal_CurvInd,FS_R_Inferiortemporal_CurvInd,FS_R_Isthmuscingulate_CurvInd,FS_R_Lateraloccipital_CurvInd,FS_R_Lateralorbitofrontal_CurvInd,FS_R_Lingual_CurvInd,FS_R_Medialorbitofrontal_CurvInd,FS_R_Middletemporal_CurvInd,FS_R_Parahippocampal_CurvInd,FS_R_Paracentral_CurvInd,FS_R_Parsopercularis_CurvInd,FS_R_Parsorbitalis_CurvInd,FS_R_Parstriangularis_CurvInd,FS_R_Pericalcarine_CurvInd,FS_R_Postcentral_CurvInd,FS_R_Posteriorcingulate_CurvInd,FS_R_Precentral_CurvInd,FS_R_Precuneus_CurvInd,FS_R_Rostralanteriorcingulate_CurvInd,FS_R_Rostralmiddlefrontal_CurvInd,FS_R_Superiorfrontal_CurvInd,FS_R_Superiorparietal_CurvInd,FS_R_Superiortemporal_CurvInd,FS_R_Supramarginal_CurvInd,FS_R_Frontalpole_CurvInd,FS_R_Temporalpole_CurvInd,FS_R_Transversetemporal_CurvInd,FS_R_Insula_CurvInd
0,100206,M,S900,1.864518e+06,1406924.0,1389007.0,1.386836e+06,304488,305634,610122,70240,807245,1239692,1225550,1225476,269963,273170,543134,2118186.0,0.754578,1.136050,32,26,58,5057,399,16808,62713,10426,4599,5941,1463,1192,1367,24537,4896,1734,1358,589,4711,...,16.7,15.0,3.2,5.3,5.0,26.1,3.7,4.2,13.4,14.1,3.5,23.4,24.5,19.4,46.6,24.6,23.3,27.1,10.6,13.6,5.7,10.2,7.5,5.5,6.1,18.7,15.6,7.8,24.3,29.8,5.6,34.6,40.1,23.3,15.6,11.9,2.9,3.8,2.8,37.5
1,100307,F,Q1,1.512540e+06,1109710.0,1098854.0,1.097122e+06,240030,243207,483238,56623,664124,957615,948872,948846,203472,203819,407292,1549276.0,0.733673,1.024287,20,46,66,3559,103,13146,61980,7787,3764,4983,1128,529,904,21307,4140,1536,869,490,3633,...,20.4,13.0,1.5,5.1,2.9,28.8,2.7,4.1,8.7,9.3,4.0,16.2,21.9,26.9,6.5,16.9,17.2,23.6,11.5,14.8,5.9,7.1,8.1,4.2,7.3,22.0,17.0,6.5,20.9,14.9,3.3,29.9,48.8,21.6,18.6,15.0,2.9,5.8,2.0,30.2
2,100408,M,Q3,1.684117e+06,1280171.0,1260943.0,1.259250e+06,260695,265243,525938,63869,726206,1106881,1092248,1091784,247504,252913,500417,1844923.0,0.760144,1.095484,36,22,58,3910,51,17022,66666,9097,3647,5622,1765,1070,2622,24481,2571,1342,1098,484,5070,...,14.7,12.5,1.4,3.7,2.6,16.7,3.2,4.1,6.5,10.5,3.1,20.2,19.0,17.6,7.6,18.0,13.5,26.8,35.1,13.8,5.0,7.9,5.1,4.0,5.8,10.1,13.4,8.3,16.4,18.4,3.3,25.2,27.5,15.2,13.9,12.3,1.8,7.8,1.9,25.1
3,100610,M,S900,1.816044e+06,1321362.0,1306929.0,1.304569e+06,276506,278002,554508,74869,762308,1155067,1144617,1144560,254742,258203,512946,1827401.0,0.727605,1.006254,16,20,36,4262,266,15463,66293,9613,4924,6690,1961,899,2076,25477,5608,1848,1141,734,5204,...,10.8,10.3,0.9,3.2,1.6,51.4,1.4,2.8,5.2,9.4,2.9,10.8,17.7,12.1,4.7,14.4,10.3,17.5,10.4,13.0,3.1,3.2,3.7,2.6,4.2,9.7,9.8,4.9,13.7,12.7,3.0,19.6,20.0,18.6,10.6,10.8,1.2,4.1,1.4,16.6
4,101006,F,S500,1.437868e+06,1053766.0,1038408.0,1.037793e+06,203718,210539,414257,58813,579632,916973,905210,903059,213012,216438,429449,1478182.0,0.732867,1.028038,9,13,22,4321,874,15105,52591,7838,3436,5726,1765,592,2184,22493,3925,1524,968,579,4665,...,8.0,7.5,0.8,2.8,1.5,9.2,1.4,5.1,4.1,6.3,2.4,9.6,12.7,7.6,6.1,20.8,8.9,11.6,5.4,8.4,1.7,2.3,3.2,2.9,4.8,7.1,9.0,4.0,15.1

##### Defining train and test data

Choose your X (train_data) and y (train_targets) as `pandas.DataFrame()` or  `numpy.array()`:

In [19]:
X = data[data.columns[3:]]
y = data[data.columns[1]]
X.shape, y.shape

((1113, 935), (1113,))

In [20]:
y.value_counts()

F    606
M    507
Name: Gender, dtype: int64

Let's change `str` values to binary classes. The easy but BAD way is:

In [21]:
y[y == 'F'] = 1
y[y == 'M'] = 0
y = y.astype(int)
y.value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1    606
0    507
Name: Gender, dtype: int64

##### Defining model grid search

 Defining new object of Grid Cross Validation Linear class:

In [22]:
from sklearn.linear_model import LogisticRegressionCV

In [23]:
# will run for 1-2 minutes
lr_cv = LogisticRegressionCV(max_iter = 1000, random_state = 42, n_jobs = -1)
lr_cv.fit(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=2000, multi_class='auto', n_jobs=-1, penalty='l2',
                     random_state=42, refit=True, scoring=None, solver='lbfgs',
                     tol=0.0001, verbose=0)

In [24]:
lr_cv.scores_

{1: array([[0.80269058, 0.78923767, 0.8206278 , 0.81165919, 0.81165919,
         0.81165919, 0.8206278 , 0.8206278 , 0.8206278 , 0.8161435 ],
        [0.76681614, 0.79820628, 0.78026906, 0.77578475, 0.78026906,
         0.77578475, 0.77578475, 0.77578475, 0.78475336, 0.78026906],
        [0.81165919, 0.84753363, 0.83856502, 0.84304933, 0.82959641,
         0.82511211, 0.82959641, 0.85201794, 0.84753363, 0.84304933],
        [0.8018018 , 0.81081081, 0.80630631, 0.81531532, 0.81531532,
         0.8018018 , 0.81531532, 0.82432432, 0.82882883, 0.82432432],
        [0.8018018 , 0.82882883, 0.82432432, 0.82432432, 0.80630631,
         0.80630631, 0.81531532, 0.81081081, 0.81981982, 0.81531532]])}

## What else?

1. Model interpretation
2. Biomarkers xploration
3. Statistical testing

How we can exploge gender morphometry biomarkers from here?

In [25]:
#lr_cv.coef_

What about feature selection?

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

In [28]:
from sklearn.feature_selection import SelectKBest, chi2
X_new = SelectKBest(k=20).fit_transform(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 55  56  57  58  98  99 100 101 174 175 176 177 178 179 180 181 293 294
 295 296 297 298 299 300 301 302 303 304] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [33]:
X.columns[SelectKBest(k=20).fit(X, y).get_support()]

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 55  56  57  58  98  99 100 101 174 175 176 177 178 179 180 181 293 294
 295 296 297 298 299 300 301 302 303 304] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Index(['FS_InterCranial_Vol', 'FS_BrainSeg_Vol', 'FS_BrainSeg_Vol_No_Vent',
       'FS_BrainSeg_Vol_No_Vent_Surf', 'FS_LCort_GM_Vol', 'FS_RCort_GM_Vol',
       'FS_TotCort_GM_Vol', 'FS_SubCort_GM_Vol', 'FS_Total_GM_Vol',
       'FS_SupraTentorial_Vol', 'FS_SupraTentorial_Vol_No_Vent',
       'FS_SupraTentorial_No_Vent_Voxel_Count', 'FS_L_WM_Vol', 'FS_R_WM_Vol',
       'FS_Tot_WM_Vol', 'FS_Mask_Vol', 'FS_L_Amygdala_Vol',
       'FS_L_Cerebellum_Cort_Vol.1', 'FS_L_Amygdala_Vox',
       'FS_R_Insula_GrayVol'],
      dtype='object')